# Overview
More or less a clone of `bold_register_scratch.ipynb` only now we are going to rely on manually alignment instead of the always failing ML models from ANTs. I don't mean to entirely imply that it is ANTs' fault, I might be misusing the models. However, it's proving difficult, and iterating over something that takes so long to align is taking forever. So lets go with the manual alignment.

**steps**
We will load all the data, run motion correction, and then manually align

# Setup and Load Data

In [1]:
from notebook_viewer_functions import *
from functions import *
from scivol import *
import numpy as np
import json
import ants
import gzip
import matplotlib.pyplot as plt
from ipywidgets import interact

proj_root = parent_directory()
print(f"project root: {proj_root}")
t1_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
bold_stim_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-emotionalfaces_run-1_bold.nii.gz")
bold_rest_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-rest_bold.nii.gz")
mni_anat_filepath =  os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_mask_filepath = os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")
events_tsv_path = os.path.join(proj_root, "media/sub-01/func/task-emotionalfaces_run-1_events.tsv")
stimulus_image_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/POFA/fMRI_POFA"
log_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/data/01-subject_emofaces1_2019_Aug_14_1903.log"

raw_t1_img = ants.image_read(t1_input_filepath)
raw_stim_bold = ants.image_read(bold_stim_filepath)
raw_rest_bold_img = ants.image_read(bold_rest_filepath)
mni_img = ants.image_read(mni_anat_filepath)
mni_mask_img = ants.image_read(mni_mask_filepath)

project root: /Users/joachimpfefferkorn/repos/neurovolume


In [2]:
bold_image = ants.image_read(bold_stim_filepath)
t1_image = ants.image_read(t1_input_filepath)

# Functions

In [3]:
#Copypasta from `bold_restister_scratch.ipynb`

def explore_fMRI(ants_img: ants.core.ants_image.ANTsImage,
                volume_override = "NULL",
                 dim="x", events_tsv="NULL",
                 cmap='nipy_spectral'):
    if type(volume_override) == np.ndarray:
        vol = volume_override
    else:
        vol = ants_img.numpy()
    
    def dim_to_indexed(dim, slice, frame):
        match dim:
            case "x":
                return vol[slice,:,:,frame]
            case "y":
                return vol[:,slice,:,frame]
            case "z":
                return vol[:,:,slice,frame]

    def plot(slice, frame):
        second = float(frame * ants_img.spacing[3])
        plt.figure()
        plt.imshow(dim_to_indexed(dim, slice, frame), cmap=cmap)
        plt.show()
        present_event = "No event file"
        if events_tsv != "NULL":
            for event in events_tsv.split("\n"):
                info = event.split("	")
                if info[0].isdigit() and info[1].isdigit():
                    if float(info[0]) <= second < float(info[0] + info[1]):
                        present_event = info[2]
            print(present_event)

    frame_slider = (0, (vol.shape[3]-1))
    match dim:
        case "x":
            interact(plot, slice=(0, vol.shape[0]-1), frame=frame_slider)
        case "y":
            interact(plot, slice=(0, vol.shape[1]-1), frame=frame_slider)
        case "z":
            interact(plot, slice=(0, vol.shape[2]-1), frame=frame_slider)

In [4]:
def compare_bold_alignment(bold_seq_vol: np.ndarray, anat_vol: np.ndarray, dim='x'):
    def x_coord(slice_idx, frame_idx, opacity):
        fig, axes = plt.subplots(1,3, figsize=(15,5))

        fig.suptitle('x axis view')

        axes[0].imshow(bold_seq_vol[slice_idx,:,:, frame_idx], cmap='hot')
        axes[0].set_title('BOLD')

        axes[1].imshow(anat_vol[slice_idx,:,:], cmap='gray')
        axes[1].set_title('Anatomy')

        axes[2].imshow(anat_vol[slice_idx,:,:], cmap='gray')
        axes[2].imshow(bold_seq_vol[slice_idx,:,:, frame_idx], cmap='hot', alpha=opacity)
        axes[2].set_title('Overlay')

    def y_coord(slice_idx, frame_idx, opacity):
        fig, axes = plt.subplots(1,3, figsize=(15,5))

        fig.suptitle('y axis view')

        axes[0].imshow(bold_seq_vol[:,slice_idx,:, frame_idx], cmap='hot')
        axes[0].set_title('BOLD')

        axes[1].imshow(anat_vol[:,slice_idx,:], cmap='gray')
        axes[1].set_title('Anatomy')

        axes[2].imshow(anat_vol[:,slice_idx,:], cmap='gray')
        axes[2].imshow(bold_seq_vol[:,slice_idx,:, frame_idx], cmap='hot', alpha=opacity)
        axes[2].set_title('Overlay')

    def z_coord(slice_idx, frame_idx, opacity):
        fig, axes = plt.subplots(1,3, figsize=(15,5))

        fig.suptitle('z axis view')

        axes[0].imshow(bold_seq_vol[:,:,slice_idx, frame_idx], cmap='hot')
        axes[0].set_title('BOLD')

        axes[1].imshow(anat_vol[:,:,slice_idx], cmap='gray')
        axes[1].set_title('Anatomy')

        axes[2].imshow(anat_vol[:,:,slice_idx], cmap='gray')
        axes[2].imshow(bold_seq_vol[:,:,slice_idx, frame_idx], cmap='hot', alpha=opacity)
        axes[2].set_title('Overlay')

    match dim:
        case "x":
            interact(x_coord, slice_idx=(0, anat_vol.shape[0]-1), frame_idx=(0, bold_seq_vol.shape[3]-1),opacity=(0, 1.0))
        case 'y':
            interact(y_coord, slice_idx=(0, anat_vol.shape[1]-1), frame_idx=(0, bold_seq_vol.shape[3]-1),opacity=(0, 1.0))
        case 'z':
            interact(z_coord, slice_idx=(0, anat_vol.shape[2]-1), frame_idx=(0, bold_seq_vol.shape[3]-1),opacity=(0, 1.0))


# The Meat of it All

In [5]:
#currently not using this:
sliced = bold_image.numpy()[:, :, :, :3]
bold_truncated_img = ants.from_numpy(sliced, spacing=bold_image.spacing, origin=bold_image.origin, direction=bold_image.direction)

In [6]:
stabilized_truncated = ants.motion_correction(bold_truncated_img)

Aha! The `compare_bold_alignment()` not work because the bold image is not registered to the t1 image. That is to say; it is not in the correct resolution. Let's do a minimal registration where we just blow it up to get the registrations the same!

In [10]:
def align_stabilized_bold_to_anat(bold_img, t1_img, template_frame_idx=0):
    """"
    This function aligns a 4D BOLD image to a T1 anatomy image
    by aligning the mean of the BOLD to the T1 anatomy.
    It uses frame registration from only one "template frame"
    as we are assuming you're using a motion corrected
    BOLD image (or something relatively stable) as your input
    
    It gets things in the ballpark, but does will require some
    extra manual registration afterwards.
    """

    print("Aligning stabilized bold to anat\nEstablishing temporal mean")
    template_frame_idx = ants.from_numpy(bold_img.numpy()[:,:,:,template_frame_idx], spacing=bold_img.spacing[:3])

    frame_registration = ants.registration(
        fixed=t1_img,
        moving=template_frame_idx,
        type_of_transform="Rigid", 
    )
    registered_frames = []

    print("Applying transformations to frames")
    for frame in range(bold_img.shape[3]):
        print(f"frame{frame}/{bold_img.shape[3]}")
        print("creating image of bold frame")
        bold_frame = ants.from_numpy(bold_img.numpy()[:,:,:,frame],
                                     spacing=bold_img.spacing[:3])
        print("     Applying frame transformation")
        registered_frame = ants.apply_transforms(
            fixed=t1_img,
            moving=bold_frame,
            transformlist=frame_registration['fwdtransforms'],
            interpolator='linear'
        )
        print("     adding registered frame to list")
        registered_frames.append(registered_frame)
    print("Creating 4D numpy vol from list of 3D ANTs imgs")
    data = np.stack([frame.numpy() for frame in registered_frames], axis=3)
    print("Creating 4D bold image from numpy vol")
    registered_bold_img = ants.from_numpy(data, origin=bold_img.origin, spacing=bold_img.spacing)
    return registered_bold_img


Just going with the truncated for now

In [8]:
registered_seq = align_stabilized_bold_to_anat(stabilized_truncated['motion_corrected'], t1_image)

Aligning stabilized bold to anat
Establishing temporal mean
Applying transformations to frames
frame0/3
creating image of bold frame
     Applying frame transformation
     adding registered frame to list
frame1/3
creating image of bold frame
     Applying frame transformation
     adding registered frame to list
frame2/3
creating image of bold frame
     Applying frame transformation
     adding registered frame to list
Creating 4D numpy vol from list of 3D ANTs imgs
Creating 4D bold image from numpy vol


In [9]:
compare_bold_alignment(registered_seq.numpy(), t1_image.numpy(), dim="z")

interactive(children=(IntSlider(value=147, description='slice_idx', max=295), IntSlider(value=1, description='…

Once this works, we'll write our manual alignment